## **CUDA STF 教程 - Part 7: 工具 (Tools)**

到目前为止，我们已经学习了 CUDA STF 的核心概念、如何定义数据和任务、各种并行构造以及模块化使用技巧。为了更好地理解、调试和优化基于 STF 的应用程序，CUDASTF 提供了一些工具和与现有工具集成的机制。

本部分主要依据您提供的文档中 "Tools" 章节 (Page 41-52) 的内容，重点介绍任务图可视化和性能分析工具。

### **14. 可视化任务图 (Visualizing task graphs)**

理解 STF 如何将您的顺序任务描述转换为并行执行图是至关重要的。CUDASTF 可以生成 [Graphviz](https://graphviz.org/) .dot 文件格式的任务图，然后您可以将其转换为图像（如 PNG、PDF）。

#### **14.1 生成基础的任务图可视化**

1. **设置环境变量**: 在运行您的 STF 应用程序之前，设置 `CUDASTF_DOT_FILE` 环境变量，指定输出的 .dot 文件名。
2. **从 .dot 文件生成图像**: 使用 Graphviz 的 `dot` 命令。

```bash
# 运行应用程序并生成 dot 文件
CUDASTF_DOT_FILE=axpy.dot ./your_stf_executable

# 生成 PDF 格式
dot -Tpdf axpy.dot -o axpy.pdf

# 生成 PNG 格式
dot -Tpng axpy.dot -o axpy.png
```

默认情况下，生成的图可能只显示高级别的任务节点，并标记为 `undefined(read)` 或 `undefined(rw)`，除非您为逻辑数据和任务添加了符号名称。

#### **14.2 使用符号名称增强可视化**

为了使任务图更具信息性，您可以为逻辑数据和任务设置符号名称 (symbols)。

**为逻辑数据命名:**
```cpp
auto lX = ctx.logical_data(X);
lX.set_symbol("MyVectorX");
// --- 或者链式调用 ---
auto lY = ctx.logical_data(Y).set_symbol("MyVectorY");
```

**为任务命名:**
```cpp
// --- 内联方式 ---
ctx.task(lX.read(), lY.rw())
   .set_symbol("AXPY_Operation")
   ->*[&](cudaStream_t s, auto dX, auto dY) { /* ... */ };

// ---显式操作任务对象 ---
auto my_task = ctx.task(lX.read(), lY.rw());
my_task.set_symbol("AXPY_Operation");
my_task->*[&](cudaStream_t s, auto dX, auto dY) { /* ... */ };
```

当您为 `parallel_for`, `launch`, `cuda_kernel` 等构造设置符号时，这个符号名称也会用于后续的 NVTX 标记，这对于性能分析很有用。

**示例：带注释的 AXPY**

让我们看一个完整的示例，展示如何添加符号名称来增强可视化效果。

In [ ]:
%%writefile p7_01_axpy_annotated.cu
#include <cuda/experimental/stf.cuh>
#include <vector>
#include <cmath>
#include <iostream>
#include <cassert>

using namespace cuda::experimental::stf;

__global__ void axpy(double a, slice<const double> x, slice<double> y) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int nthreads = gridDim.x * blockDim.x;

    for (int i = tid; i < x.size(); i += nthreads) {
        y(i) += a * x(i);
    }
}

double X0(int i) { return sin((double)i); }
double Y0(int i) { return cos((double)i); }

int main() {
    context ctx;
    const size_t N = 16;
    double X[N], Y[N];

    for (size_t i = 0; i < N; i++) {
        X[i] = X0(i);
        Y[i] = Y0(i);
    }

    double alpha = 3.14;

    // 为逻辑数据设置符号名称
    auto lX = ctx.logical_data(X).set_symbol("X");
    auto lY = ctx.logical_data(Y).set_symbol("Y");

    /* Compute Y = Y + alpha X */
    ctx.task(lX.read(), lY.rw())
        .set_symbol("axpy")  // 为任务设置符号名称
        ->*[&](cudaStream_t s, auto dX, auto dY) {
            axpy<<<16, 128, 0, s>>>(alpha, dX, dY);
        };

    ctx.finalize();

    for (size_t i = 0; i < N; i++) {
        assert(fabs(Y[i] - (Y0(i) + alpha * X0(i))) < 0.0001);
        assert(fabs(X[i] - X0(i)) < 0.0001);
    }
    
    std::cout << "Annotated AXPY example completed successfully!" << std::endl;
    return 0;
}

In [ ]:
# 编译带注释的 AXPY 示例
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include p7_01_axpy_annotated.cu -o p7_01_axpy_annotated -arch=sm_86 -lcuda

In [ ]:
# 运行程序并生成任务图
!CUDASTF_DOT_FILE=axpy_annotated.dot ./p7_01_axpy_annotated

In [ ]:
# 检查是否安装了 graphviz
!which dot || echo "需要安装 graphviz: apt install graphviz"

In [ ]:
# 从 dot 文件生成可视化图像
!dot -Tpng axpy_annotated.dot -o axpy_annotated.png
!echo "任务图已生成: axpy_annotated.png"

#### **14.3 高级可视化选项**

CUDASTF 提供了多个环境变量来控制生成的任务图的细节级别和外观：

**1. CUDASTF_DOT_IGNORE_PREREQS=0**

默认情况下，STF 可能会隐藏一些内部生成的异步操作（如内存分配、拷贝）。将此变量设置为 0 可以显示这些更底层的操作，从而提供一个更完整的依赖图。

**2. CUDASTF_DOT_COLOR_BY_DEVICE**

如果设置为非空值，任务节点将根据其执行设备进行着色。这对于理解多 GPU 应用中的任务分布非常有用。

**3. CUDASTF_DOT_REMOVE_DATA_DEPS**

如果设置为非空值，将从任务节点标签中移除数据依赖列表，以简化图形显示，专注于任务流。

**4. CUDASTF_DOT_TIMING**

如果设置为非空值，将在图中包含计时信息。节点会根据其相对持续时间着色，并且测量的持续时间会包含在任务标签中。

In [ ]:
# 生成包含更多细节的任务图（显示内部操作）
!CUDASTF_DOT_IGNORE_PREREQS=0 CUDASTF_DOT_FILE=axpy_detailed.dot ./p7_01_axpy_annotated
!dot -Tpng axpy_detailed.dot -o axpy_detailed.png
!echo "详细任务图已生成: axpy_detailed.png"

In [ ]:
# 生成带计时信息的任务图
!CUDASTF_DOT_TIMING=1 CUDASTF_DOT_FILE=axpy_timing.dot ./p7_01_axpy_annotated
!dot -Tpng axpy_timing.dot -o axpy_timing.png
!echo "带计时的任务图已生成: axpy_timing.png"

#### **14.4 结构化和浓缩图表可视化 - Dot Sections**

对于包含成千上万个任务的实际工作负载，直接使用 dot 生成的图可能过于庞大且难以阅读。CUDASTF 允许使用 **点段 (dot sections)** 来结构化图表。

**创建点段**: 通过 `ctx.dot_section("section_name")` 创建一个 dot_section 对象。该段的范围从对象创建开始，直到对象被销毁（RAII 风格）或显式调用其 `end()` 方法。点段可以嵌套。

In [ ]:
%%writefile p7_02_dot_sections.cu
#include <cuda/experimental/stf.cuh>
#include <iostream>
#include <string>

using namespace cuda::experimental::stf;

int main() {
    context ctx;
    
    // 创建一些令牌用于演示
    auto lA = ctx.token().set_symbol("A");
    auto lB = ctx.token().set_symbol("B");
    auto lC = ctx.token().set_symbol("C");

    // 初始化任务
    ctx.task(lA.write()).set_symbol("initA")->*[](cudaStream_t) {};
    ctx.task(lB.write()).set_symbol("initB")->*[](cudaStream_t) {};
    ctx.task(lC.write()).set_symbol("initC")->*[](cudaStream_t) {};

    // 开始外部段
    auto s_outer = ctx.dot_section("Outer_Calculation");
    
    for (int i = 0; i < 2; ++i) {
        // 内部段使用 RAII
        auto s_inner = ctx.dot_section("Inner_Loop_Iter_" + std::to_string(i));
        
        ctx.task(lA.read(), lB.rw())
            .set_symbol("task_in_inner_" + std::to_string(i))
            ->*[](cudaStream_t, auto, auto) {};
            
        // 嵌套更深的段
        {
            auto s_deep = ctx.dot_section("Deep_Section_" + std::to_string(i));
            ctx.task(lB.read(), lC.rw())
                .set_symbol("deep_task_" + std::to_string(i))
                ->*[](cudaStream_t, auto, auto) {};
        } // s_deep 在此销毁，深层段结束
        
    } // s_inner 在此销毁，内部段结束
    
    s_outer.end(); // 显式结束外部段
    
    // 最终任务
    ctx.task(lA.read(), lB.read(), lC.rw())
        .set_symbol("final_task")
        ->*[](cudaStream_t, auto, auto, auto) {};

    ctx.finalize();
    
    std::cout << "Dot sections example completed!" << std::endl;
    return 0;
}

In [ ]:
# 编译 dot sections 示例
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include p7_02_dot_sections.cu -o p7_02_dot_sections -arch=sm_86 -lcuda

In [ ]:
# 生成带段结构的任务图
!CUDASTF_DOT_FILE=sections.dot ./p7_02_dot_sections
!dot -Tpng sections.dot -o sections.png
!echo "带段结构的任务图已生成: sections.png"

**CUDASTF_DOT_MAX_DEPTH**: 通过设置此环境变量，可以控制生成图中显示的嵌套深度。任何嵌套级别超过指定值的段和任务都将被折叠显示。

In [ ]:
# 生成不同深度的任务图
!CUDASTF_DOT_MAX_DEPTH=0 CUDASTF_DOT_FILE=sections_depth0.dot ./p7_02_dot_sections
!dot -Tpng sections_depth0.dot -o sections_depth0.png
!echo "深度0任务图已生成: sections_depth0.png"

!CUDASTF_DOT_MAX_DEPTH=1 CUDASTF_DOT_FILE=sections_depth1.dot ./p7_02_dot_sections
!dot -Tpng sections_depth1.dot -o sections_depth1.png
!echo "深度1任务图已生成: sections_depth1.png"

### **15. 使用 ncu 进行核函数性能分析 (Kernel tuning with ncu)**

NVIDIA Nsight Compute (ncu) 是一个强大的工具，用于分析 CUDA 核函数的性能。您可以将 ncu 与使用 `ctx.parallel_for` 和 `ctx.launch` (以及其他提交核函数的 STF 构造) 生成的核函数一起使用。

#### **15.1 命名核函数以便 ncu 识别**

默认情况下，由 `parallel_for` 或 `launch` 生成的核函数在 ncu 中的名称可能不够具有描述性 (例如，都显示为 `thrust::cuda_cub::core::_kernel_agent`)。

为了解决这个问题，您应该使用任务的 `set_symbol("YourKernelName")` 方法。STF 会使用这个符号名称来创建 NVTX (NVIDIA Tools Extension) 范围，ncu 可以利用这些 NVTX 注释来重命名其报告中的核函数。

In [ ]:
%%writefile p7_03_parallel_for_profiling.cu
#include <cuda/experimental/stf.cuh>
#include <iostream>
#include <vector>

using namespace cuda::experimental::stf;

int main() {
    context ctx;
    const size_t N = 1000000;  // 更大的数据集用于性能分析
    
    // 创建数据
    std::vector<double> h_x(N), h_y(N);
    for (size_t i = 0; i < N; i++) {
        h_x[i] = sin(double(i));
        h_y[i] = cos(double(i));
    }
    
    auto lX = ctx.logical_data(h_x).set_symbol("VectorX");
    auto lY = ctx.logical_data(h_y).set_symbol("VectorY");
    auto lsum = ctx.logical_data(shape_of<scalar_view<double>>()).set_symbol("DotSum");
    
    double alpha = 2.5;
    
    // AXPY 操作：Y = Y + alpha * X
    ctx.parallel_for(lX.shape(), lX.read(), lY.rw())
        .set_symbol("AXPY_Kernel")  // 为核函数设置符号名称
        ->*[] __device__(size_t i, auto dX, auto dY) {
            dY(i) += 2.5 * dX(i);
        };
    
    // 点积计算：sum(X[i] * Y[i])
    ctx.parallel_for(lX.shape(), lX.read(), lY.read(), lsum.reduce(reducer::sum<double>{}))
        .set_symbol("DotProduct_Kernel")  // 为核函数设置符号名称
        ->*[] __device__(size_t i, auto dX, auto dY, double& sum) {
            sum += dX(i) * dY(i);
        };
    
    double result = ctx.wait(lsum);
    ctx.finalize();
    
    std::cout << "Dot product result: " << result << std::endl;
    std::cout << "Profiling example completed!" << std::endl;
    return 0;
}

In [ ]:
# 编译性能分析示例（使用优化标志）
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -O3 -I../cccl/libcudacxx/include -I../cccl/cudax/include p7_03_parallel_for_profiling.cu -o p7_03_parallel_for_profiling -arch=sm_86 -lcuda

In [ ]:
# 先运行一次确保程序正常工作
!./p7_03_parallel_for_profiling

#### **15.2 运行 ncu 进行性能分析**

一般的 ncu 命令流程如下：

1. **编译优化后的代码**: 性能分析应始终在优化构建上进行。
2. **使用 ncu 运行应用程序**:

```bash
ncu --section=ComputeWorkloadAnalysis --print-nvtx-rename kernel --nvtx -o output_profile your_stf_executable [args...]
```

参数说明：
- `--section=ComputeWorkloadAnalysis`: 选择要收集的性能指标部分
- `--print-nvtx-rename kernel`: 指示 ncu 根据 NVTX 范围重命名内核
- `--nvtx`: 启用 NVTX 跟踪
- `-o output_profile`: 指定输出报告文件的名称

**注意**: 根据您的机器配置，您可能需要以 root 用户身份运行 ncu 或进行特定设置以允许访问 NVIDIA GPU 性能计数器。

In [ ]:
# 检查 ncu 是否可用
!which ncu || echo "ncu 未找到，请确保安装了 NVIDIA Nsight Compute"

In [ ]:
# 使用 ncu 进行性能分析（可能需要 root 权限）
# 注意：在某些环境中可能需要特殊权限或配置
!ncu --section=ComputeWorkloadAnalysis --print-nvtx-rename kernel --nvtx -o stf_profile ./p7_03_parallel_for_profiling || echo "ncu 分析失败，可能需要特殊权限或配置"

#### **15.3 使用 nsys 进行系统级性能分析**

除了 ncu，我们还可以使用 NVIDIA Nsight Systems (nsys) 进行系统级的性能分析，这对于理解整体应用程序的行为很有用。

In [ ]:
# 使用 nsys 进行系统级性能分析
!nsys profile --stats=true -t nvtx,cuda --cuda-event-trace=false --force-overwrite=true -o stf_nsys_profile ./p7_03_parallel_for_profiling

### **16. 实际示例：分析复杂的 STF 应用**

让我们使用 CCCL 提供的一个更复杂的示例来演示这些工具的实际应用。

In [ ]:
# 编译并分析 dot-reduce 示例
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -O3 -I../cccl/libcudacxx/include -I../cccl/cudax/include ../cccl/cudax/examples/stf/09-dot-reduce.cu -o p7_dot_reduce -arch=sm_86 -lcuda

In [ ]:
# 运行并生成任务图
!CUDASTF_DOT_FILE=dot_reduce.dot ./p7_dot_reduce
!dot -Tpng dot_reduce.dot -o dot_reduce.png
!echo "Dot-reduce 任务图已生成: dot_reduce.png"

In [ ]:
# 生成详细的任务图
!CUDASTF_DOT_IGNORE_PREREQS=0 CUDASTF_DOT_FILE=dot_reduce_detailed.dot ./p7_dot_reduce
!dot -Tpng dot_reduce_detailed.dot -o dot_reduce_detailed.png
!echo "详细的 dot-reduce 任务图已生成: dot_reduce_detailed.png"

### **动手试试:**

1. **编译并运行本部分提供的示例**，观察生成的任务图的差异。

2. **尝试不同的可视化选项**：
   - 比较使用和不使用 `CUDASTF_DOT_IGNORE_PREREQS=0` 的图
   - 尝试 `CUDASTF_DOT_TIMING=1` 来查看计时信息
   - 使用不同的 `CUDASTF_DOT_MAX_DEPTH` 值

3. **研究 CCCL 提供的其他示例**：
   - 尝试编译和可视化 `../cccl/cudax/examples/stf/` 目录下的其他示例
   - 特别关注那些使用了 `parallel_for` 和 `launch` 的示例

4. **性能分析实践**：
   - 如果您的环境支持，尝试使用 ncu 分析示例程序
   - 使用 nsys 进行系统级分析
   - 观察 NVTX 标记如何帮助识别不同的核函数

5. **创建自己的示例**：
   - 修改现有示例，添加更多的符号名称和 dot sections
   - 尝试创建一个包含多个循环和嵌套段的复杂示例

### **教程总结**

恭喜您完成了这个 CUDA STF 学习教程！我们从 STF 的基本概念开始，逐步学习了：

**Part 1-3: 基础概念**
- **核心组件**: scheduler, context (包括 stream_ctx 和 graph_ctx)
- **数据管理**: logical_data, slice, 从形状创建数据，写回策略
- **任务创建**: ctx.task(), 数据依赖声明 (read, write, rw, reduce)

**Part 4: 同步与执行**
- **同步机制**: finalize(), submit(), task_fence(), wait()
- **位置管理**: exec_place 和 data_place

**Part 5: 高级并行构造**
- **parallel_for**: 包括 box 形状和 reduce 模式
- **launch**: 包括线程层级

**Part 6: 专业特性**
- **直接核函数调用**: ctx.cuda_kernel(), ctx.cuda_kernel_chain()
- **类型系统**: STF 如何利用 C++ 类型来增强代码的健壮性
- **模块化使用**: 冻结数据和令牌

**Part 7: 工具与调试**
- **任务图可视化**: 使用 Graphviz 和各种环境变量选项
- **结构化可视化**: dot sections 和深度控制
- **性能分析**: 使用 ncu 和 nsys 进行核函数和系统级分析

**后续学习建议：**

1. **实践更多示例**: 遍历 `../cccl/cudax/examples/stf/` 目录下的所有示例，特别是那些涉及更复杂算法（如图算法、线性代数）或多 GPU 的示例。

2. **查阅官方文档**: 对于任何不清楚的 API 或概念，[CUDA STF 官方文档](https://nvidia.github.io/cccl/cudax/stf.html) 始终是最终的参考。

3. **尝试修改示例**: 不要只是运行它们。尝试修改数据大小、算法参数、任务结构，观察性能和行为的变化。

4. **构建自己的小程序**: 尝试使用 STF 来实现一些您熟悉的小型并行算法，这将是巩固知识的最佳方式。

5. **关注性能**: 当您熟悉了 STF 的功能后，开始关注性能。使用 ncu 和任务图可视化来识别瓶颈并进行优化。考虑不同的上下文后端 (stream_ctx vs graph_ctx) 对性能的影响。

6. **探索高级特性**: 研究多 GPU 编程、自定义数据接口、以及与其他 CUDA 库的集成。

CUDA STF 是一个强大的工具，可以帮助您更轻松地开发复杂的高性能 CUDA 应用程序。希望本教程对您的学习有所帮助！祝您在并行编程的道路上一切顺利！